<a href="https://colab.research.google.com/github/Mohammed-Jemal/Product_recomendation_model/blob/main/AI4ALL_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.cluster import KMeans
from scipy.sparse import hstack
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

##Since it doesn’t contain direct user ratings, we’ll focus on a content-based recommendation system, where we recommend products similar to those a user has already shown interest in. This method will leverage the product’s categorical and textual data (like "Product Specification" and "Technical Details")

In [2]:
df=pd.read_csv('/content/drive/MyDrive/data.csv')

In [ ]:
df.head(5)

,Uniq Id,Product Name,Brand Name,Asin,Category,Upc Ean Code,List Price,Selling Price,Quantity,Model Number,...,Product Url,Stock,Product Details,Dimensions,Color,Ingredients,Direction To Use,Is Amazon Seller,Size Quantity Variant,Product Description
0,4c69b61db1fc16e7013b43fc926e502d,"DB Longboards CoreFlex Crossbow 41"" Bamboo Fib...",NaN,NaN,Sports & Outdoors | Outdoor Recreation | Skate...,NaN,NaN,$237.68,NaN,NaN,...,https://www.amazon.com/DB-Longboards-CoreFlex-...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
1,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,NaN,Toys & Games | Learning & Education | Science ...,NaN,NaN,$99.95,NaN,55324,...,https://www.amazon.com/Electronic-Circuits-Cla...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
2,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,NaN,Toys & Games | Arts & Crafts | Craft Kits,NaN,NaN,$34.99,NaN,NaN,...,https://www.amazon.com/3Doodler-Plastic-Innova...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
3,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,NaN,Toys & Games | Hobbies | Models & Model Kits |...,NaN,NaN,$28.91,NaN,142,...,https://www.amazon.com/Guillow-Airplane-Design...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN
4,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,NaN,NaN,$17.49,NaN,62151,...,https://www.amazon.com/Woodstock-Collage-500-p...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN


In [61]:
df.isna().sum(),df.shape

(Uniq Id                      0
 Product Name                 0
 Brand Name               10002
 Asin                     10002
 Category                   830
 Upc Ean Code              9968
 List Price               10002
 Selling Price              107
 Quantity                 10002
 Model Number              1772
 About Product              273
 Product Specification     1632
 Technical Details          790
 Shipping Weight           1138
 Product Dimensions        9523
 Image                        0
 Variants                  7524
 Sku                      10002
 Product Url                  0
 Stock                    10002
 Product Details          10002
 Dimensions               10002
 Color                    10002
 Ingredients              10002
 Direction To Use         10002
 Is Amazon Seller             0
 Size Quantity Variant    10002
 Product Description      10002
 dtype: int64,
 (10002, 28))

In [42]:
df['Product Name'].unique()[10:20]

array(['Moonlite, 3-Story Bundle for Boys, Includes 3 Story Reels for Use with Storybook Projector',
       'DC Cover Girls: Black Canary by Joëlle Jones Statue',
       'Baby Doll Bedding Chevron Window Valance and Curtain Set, Orange',
       'LEGO Minecraft Creeper BigFig and Ocelot Characters 21156 Buildable Toy Minecraft Figure Gift Set for Play and Decoration, New 2020 (184 Pieces)',
       'C&D Visionary DC Comic Originals Flash Logo Sticker',
       'The Learning Journey ABC Melody Maker, Primary',
       'The Complete Common Core: State Standards Kit, Grade 5',
       "Flash Furniture 25''W x 45''L Trapezoid Red HP Laminate Activity Table - Height Adjustable Short Legs",
       'Schylling Veterinarian Kit',
       'Yellies! Frizz; Voice-Activated Spider Pet; Ages 5 & Up'],
      dtype=object)

In [4]:
df['Category'].unique()[:20]

array(['Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Skateboarding | Standard Skateboards & Longboards | Longboards',
       'Toys & Games | Learning & Education | Science Kits & Toys',
       'Toys & Games | Arts & Crafts | Craft Kits',
       'Toys & Games | Hobbies | Models & Model Kits | Model Kits | Airplane & Jet Kits',
       'Toys & Games | Puzzles | Jigsaw Puzzles', nan,
       'Clothing, Shoes & Jewelry | Costumes & Accessories | Kids & Baby | Girls | Costumes',
       'Toys & Games | Arts & Crafts | Drawing & Painting Supplies | Crayons',
       'Home & Kitchen | Home Décor | Window Treatments | Window Stickers & Films | Window Films',
       'Toys & Games | Baby & Toddler Toys',
       'Toys & Games | Collectible Toys | Statues, Bobbleheads & Busts | Statues',
       'Baby Products | Nursery | Décor | Window Treatments | Valances',
       'Toys & Games | Building Toys | Building Sets',
       'Toys & Games | Baby & Toddler Toys | Musical Toys',


In [5]:
df['Selling Price'][:5]

,Selling Price
0,$237.68
1,$99.95
2,$34.99
3,$28.91
4,$17.49


In [ ]:
df['Shipping Weight'].unique()[:10]

array(['10.7 pounds', '4 pounds', '12.8 ounces', '13.4 ounces',
       '1.4 pounds', '9.8 ounces', nan, '12.2 ounces', '1.5 pounds',
       '3.2 ounces'], dtype=object)

In [ ]:
df['Technical Details'].unique()

array([nan,
       'The snap circuits mini kits classpack provides basic electric circuitry information for students in grades 2-6. This classpack includes 5 snap-together building kits. Components snap together to create working models of everyday electronic devices. Kits included are an FM radio, a motion detector, a music box, space battle sound effects, and a flying saucer. Each mini kit comes with individual components, and an activity guide which includes instructions and additional project ideas. Each primary-colored component represents one function in a circuit. Activity kits are used by teachers and students in classroom and homeschool settings for educational and research applications in science, math, and for a variety of additional disciplines. Science education products and manipulatives incorporate applied math and science principles into classroom or homeschool projects. Teachers in pre-K, elementary, and secondary classrooms use science education kits, manipualtives, a

In [7]:
Selected_df= df[['Product Name','Category','Selling Price','Product Specification','Technical Details','Shipping Weight','Is Amazon Seller']]

#Handling the Selling price feature with median values

In [8]:
Selected_df.loc[:,'Selling Price'] =Selected_df['Selling Price'].astype('str').str.replace(r'[^\d.]', '',regex=True)
Selected_df.loc[:,'Selling Price']= pd.to_numeric(Selected_df['Selling Price'],errors='coerce')
imputer= SimpleImputer(strategy='median')
Selected_df.loc[:,'Selling Price']= imputer.fit_transform(Selected_df[['Selling Price']])


In [9]:
Selected_df.isna().sum()

,0
Product Name,0
Category,830
Selling Price,0
Product Specification,1632
Technical Details,790
Shipping Weight,1138
Is Amazon Seller,0


In [10]:
# Function to convert weight to ounces with additional checks
def to_ounces(weight):
    if pd.isna(weight):
        return np.nan
    weight_str = str(weight).strip().lower()

    try:
        if 'pounds' in weight_str:
            return float(weight_str.split()[0]) * 16
        elif 'ounces' in weight_str:
            return float(weight_str.split()[0])
        else:
            return np.nan
    except ValueError:
        return np.nan

# Apply the conversion to the 'Shipping Weight' column
Selected_df.loc[:,'Shipping Weight'] = Selected_df['Shipping Weight'].apply(to_ounces)

imputer = SimpleImputer(strategy='median')
Selected_df['Shipping Weight'] = imputer.fit_transform(Selected_df[['Shipping Weight']])


<ipython-input-10-ea61031d69cc>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Selected_df['Shipping Weight'] = imputer.fit_transform(Selected_df[['Shipping Weight']])


In [11]:
#Handling missing value
Selected_df.loc[:,'Technical Details']= Selected_df['Technical Details'].fillna('No detail available')

#Handling categorical feature

In [12]:
#Droping missing value from category
Selected_df = Selected_df.dropna(subset=['Category'])

In [13]:
#Droping missing value from product specification
Selected_df= Selected_df.dropna(subset=['Product Specification'])

In [14]:
print('Cleaned features:')
Selected_df.isna().sum()

Cleaned features:


,0
Product Name,0
Category,0
Selling Price,0
Product Specification,0
Technical Details,0
Shipping Weight,0
Is Amazon Seller,0


In [54]:
# Combine features for content-based filtering
Selected_df['combined'] = Selected_df['Product Specification'] + " " + Selected_df['Technical Details'] + " " + Selected_df['Category']

# Transform the combined text features into numerical vectors using TF-IDF
tfid = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 2))

tfidf_matrix = tfid.fit_transform(Selected_df['combined']).tocsr()

# Normalize numerical features (Selling Price and Shipping Weight)
scale = MinMaxScaler()
Selected_df[['Selling Price', 'Shipping Weight']] = scale.fit_transform(Selected_df[['Selling Price', 'Shipping Weight']])

# Combine the normalized numerical features with the TF-IDF matrix

numeric_features = Selected_df[['Selling Price', 'Shipping Weight']].values  # Extract normalized features as array
tfidf_matrix = hstack([tfidf_matrix, numeric_features])  # Combine TF-IDF matrix with numerical features

tfidf_matrix = tfidf_matrix.tocsr()

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Apply KMeans clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
Selected_df['cluster'] = kmeans.fit_predict(tfidf_matrix)  # Use `fit_predict` instead of `fit_transform`

# Create a series to map each product name to its index
product_index = pd.Series(Selected_df.index, index=Selected_df['Product Name']).drop_duplicates()


#Recommendation function

In [58]:
def recommend_with_clustering(product_name, Selected_df=Selected_df, n_recommendations=10):
    # Get the cluster of the input product
    product_cluster = Selected_df.loc[Selected_df['Product Name'] == product_name, 'cluster'].values[0]

    # Filter products within the same cluster
    cluster_products = Selected_df[Selected_df['cluster'] == product_cluster]

    # Compute similarity within the cluster
    idx = Selected_df[Selected_df['Product Name'] == product_name].index[0]

    # Convert `cluster_products.index` to match rows in `tfidf_matrix`
    cluster_indices = [i for i in cluster_products.index if i < tfidf_matrix.shape[0]]

    sim_scores = list(enumerate(cosine_similarity(tfidf_matrix[idx].reshape(1, -1), tfidf_matrix[cluster_indices]).flatten()))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get indices of the top recommendations
    recommendations = cluster_products.iloc[[i[0] for i in sim_scores[1:n_recommendations + 1]]]
    return recommendations['Product Name']




In [66]:
# Example usage
#product_name = "Guillow Airplane Design Studio with Travel Case Building Kit"
product_name= 'The Learning Journey ABC Melody Maker, Primary'

recommendations = recommend_with_clustering(product_name)
print("Recommendations for:", product_name)
print(recommendations)


Recommendations for: The Learning Journey ABC Melody Maker, Primary
1617           Easy Model T-34/85 Egyptian Army Model Kit
356     Toysmith Get Outside GO! Neon Dart Ball Set, P...
4567    Duck-A-Roo! Kids Memory Game in A Duck-Shaped Box
574     My Little Pony Friendship is Magic Fluttershy ...
688     Education Outdoors Fawn Pet Pod, One Color, On...
746     Shopkins Strawberry Kiss Lil' Characters Shade...
262     Avengers Marvel Black Panther 6"-Scale Marvel ...
6029    PlayTape 2” Tight Curves 8 Pack – Road Car Tap...
4823    Power Rangers Ninja Steel Training Gear - Doub...
4081    Melissa & Doug Birthday Party Cake (Wooden Pla...
Name: Product Name, dtype: object
